In [66]:
import os
import requests
from urllib.request import urlopen
import pathlib
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing
import re
from collections import namedtuple
# For pausing
from time import sleep
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_251.jdk/Contents/Home"
import pickle
from itertools import repeat
import psycopg2
import pandas.io.sql as sqlio
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
from pymetamap import MetaMap

import json
import argparse
import lxml

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [108]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)
soup = BeautifulSoup(response.text)
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects
print(upload_dates)

latest_file_date = max(upload_dates)
latest_file_date = latest_file_date.strftime('%Y%m%d') 
print(latest_file_date)


[datetime.date(2022, 7, 12), datetime.date(2022, 7, 11), datetime.date(2022, 7, 7), datetime.date(2022, 7, 6), datetime.date(2022, 7, 5), datetime.date(2022, 7, 4), datetime.date(2022, 7, 3), datetime.date(2022, 7, 2), datetime.date(2022, 7, 1), datetime.date(2022, 6, 30), datetime.date(2022, 6, 29), datetime.date(2022, 6, 28), datetime.date(2022, 6, 27), datetime.date(2022, 6, 26), datetime.date(2022, 6, 25), datetime.date(2022, 6, 24), datetime.date(2022, 6, 23), datetime.date(2022, 6, 22), datetime.date(2022, 6, 21), datetime.date(2022, 6, 20), datetime.date(2022, 6, 18), datetime.date(2022, 6, 16), datetime.date(2022, 6, 15), datetime.date(2022, 6, 14), datetime.date(2022, 6, 13), datetime.date(2022, 6, 12), datetime.date(2022, 6, 11), datetime.date(2022, 6, 10), datetime.date(2022, 6, 9), datetime.date(2022, 6, 8), datetime.date(2022, 6, 7), datetime.date(2022, 6, 6), datetime.date(2022, 6, 5), datetime.date(2022, 6, 4), datetime.date(2022, 6, 3), datetime.date(2022, 6, 2), dateti

In [110]:
import urllib
import zipfile
url = "https://aact.ctti-clinicaltrials.org/static/exported_files/daily/{}_pipe-delimited-export.zip".format(latest_file_date)
data_dir = "{}/data".format(pathlib.Path.cwd().parents[0])
data_extracted = data_dir + "/{}_extracted".format(latest_file_date)
data_path = pathlib.Path.cwd().parents[0] / "{}/{}_pipe-delimited-export.zip".format(data_dir, latest_file_date)
# print(data_path)
if not os.path.exists(data_path):
    req = requests.get(url)
    with open(data_path, 'wb') as download:
        download.write(req.content)      
    
#     driver()
else:
    print("KG is already up to date.")
    
print("Finished download")




Finished download


In [111]:
with zipfile.ZipFile(data_path, 'r') as download:
    download.extractall(data_extracted) 

In [112]:

conditions_df = pd.read_csv(data_extracted + '/conditions.txt', sep='|', index_col=False, header=0)
interventions_df = pd.read_csv(data_extracted + '/interventions.txt', sep='|', index_col=False, header=0)
browse_conditions_df = pd.read_csv(data_extracted + '/browse_conditions.txt', sep='|', index_col=False, header=0)
browse_interventions_df = pd.read_csv(data_extracted + '/browse_interventions.txt', sep='|', index_col=False, header=0)


In [113]:
conditions_df

,id,nct_id,intervention_type,name,description
0,14315260,NCT00821093,Drug,Indacaterol 150 µg,Indacaterol 150 μg was provided in powder-fill...
1,13885110,NCT01911611,Drug,RO6870868,Single ascending doses
2,13885111,NCT01911611,Drug,placebo,Single ascending doses
3,13885112,NCT01911624,Drug,direct thrombin inhibition,NaN
4,14315261,NCT00821093,Drug,Salmeterol 50 µg,Salmeterol 50 μg was provided in the manufactu...
...,...,...,...,...,...
716917,14314647,NCT00807248,Drug,Escitalopram 20 mg,Once daily before bedtime for 8 weeks
716918,14314648,NCT00807248,Drug,Gaboxadol 5 mg,Once daily before bedtime for 8 weeks
716919,14314649,NCT00807248,Drug,Gaboxadol 10 mg,Once daily before bedtime for 8 weeks
716920,14314650,NCT00807456,Device,"ASTRA TECH Implant System, OsseoSpeed™","ASTRA TECH Implant System, OsseoSpeed™ Profile..."
